# Style Transfer with Deep Neural Networks


In this notebook, I’ll *recreate* a style transfer method that is outlined in the paper, [Image Style Transfer Using Convolutional Neural Networks, by Gatys](https://www.cv-foundation.org/openaccess/content_cvpr_2016/papers/Gatys_Image_Style_Transfer_CVPR_2016_paper.pdf) in PyTorch.

In this paper, style transfer uses the features found in the 19-layer VGG Network, which is comprised of a series of convolutional and pooling layers, and a few fully-connected layers.

### Separating Style and Content

Style transfer relies on separating the content and style of an image. Given one content image and one style image, we aim to create a new _target_ image which should contain our desired content and style components:
* objects and their arrangement are similar to that of the **content image**
* style, colors, and textures are similar to that of the **style image**

In this notebook, I'll use a pre-trained VGG19 Net to extract content or style features from an image.

In [ ]:
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
from os import path
import pickle

import cv2
import torch
from torchvision import models

from helper import *
from transfer import *

%matplotlib inline

### Setup wheter I am using Google Colab or not

In [ ]:
##### Change the next variable to False if you are not using Google Colab
using_colab = False
#########################################################################

if using_colab:
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)
    drive_path = '/content/drive/My Drive/Colab Notebooks/'

### Pick input video and style to be transfered

In [ ]:
video_file_name = 'mendelson.mp4'
style_file_name = 'starry_night.jpg'

### I will use the VGG19 pretrained model

In [ ]:
from ipywidgets import IntProgress

# get the "features" portion of VGG19 (we will not need the "classifier" portion)
vgg = models.vgg19(pretrained=True).features

# freeze all VGG parameters since I'm only optimizing the target image
for param in vgg.parameters():
    param.requires_grad_(False)

### The following cell checks if there is a GPU available

In [ ]:
# move the model to GPU, if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(f'===== Using {device} =====')

vgg = vgg.to(device);

if torch.cuda.is_available():
    print(f'GPU in use: {torch.cuda.get_device_name(0)}')

### Let's see the style picture

In [ ]:
# Style to be applied
style_file_name = 'styles/' + style_file_name
if using_colab:
    style_file_name = drive_path + 'frames/' + style_file_name

style = load_image(style_file_name).to(device)
plt.imshow(im_convert(style))
plt.show()

### Only run the following cell if you don't have the video frames

This cell extracts the frames from the input video and save them to disk.

In [ ]:
img_array = []
 
# Opens the Video file
cap = cv2.VideoCapture('input videos/' + video_file_name)

# https://docs.opencv.org/2.4/modules/highgui/doc/reading_and_writing_images_and_video.html
# https://stackoverflow.com/questions/39953263/get-video-dimension-in-python-opencv/39953739
width = int(cap.get(3))
height = int(cap.get(4))
fps = round(cap.get(5))
total_frames = int(cap.get(7))

# Saving info to file
with open('properties.pkl', 'wb') as f:
    pickle.dump([width, height, fps, total_frames], f)

current_frame_number = 0

while(cap.isOpened()):
    ret, frame = cap.read()
    if ret == False:
        break
    img_array.append(frame)
    
    file_name = 'frames/input frames/input_frame_{:08d}.jpg'.format(current_frame_number)
    matplotlib.image.imsave(file_name, img_array[-1])
    current_frame_number += 1

cap.release()
cv2.destroyAllWindows()
img_array.clear()

### Reading frames saved to disk from the input video.

In [ ]:
# Loading frames from files
with open('properties.pkl', 'rb') as f:
    width, height, fps, total_frames = pickle.load(f)
#     height, width, fps, total_frames = pickle.load(f)

input_frames = []

current_frame_number = 0

file_name = 'frames/input frames/input_frame_{:08d}.jpg'.format(current_frame_number)

if using_colab:
    file_name = drive_path + file_name
    
while path.exists(file_name):
    input_frames.append(load_image(file_name).to(device))
    current_frame_number += 1
    file_name = 'frames/input frames/input_frame_{:08d}.jpg'.format(current_frame_number)
    if using_colab:
        file_name = drive_path + file_name

### If you already have the stylized frames, skip the next cell

This cell performs the style transfer on each frame and save it to disk.

In [ ]:
# Transfering style and saving to file
for idx, image in enumerate(input_frames):
    print(f'Currently evaluating frame {idx + 1} of {total_frames}')
    frame = transfer_to_frame(image, style, vgg, device)
    
    file_name = 'frames/style frames/stylized_frame_{:08d}.jpg'.format(idx)
    if using_colab:
        file_name = drive_path + file_name
        
    current = frame.to('cpu').detach()
    matplotlib.image.imsave(file_name, im_convert(current))

### Reading stylized frames from the disk

In [ ]:
stylized_frames = []

# Load stylized frames from file
current_frame_number = 0

file_name = 'frames/style frames/stylized_frame_{:08d}.jpg'.format(current_frame_number)
if using_colab:
    file_name = drive_path + file_name

while path.exists(file_name):
    stylized_frames.append(load_image(file_name).to(device))
    current_frame_number += 1
    
    file_name = 'frames/style frames/stylized_frame_{:08d}.jpg'.format(current_frame_number)
    if using_colab:
        file_name = drive_path + file_name


### Prepare the frames and join them into a sequence of frames

In [ ]:
final_frames = []

# Convert tensors back to numpy arrays
for tensor_frame in stylized_frames:
    temp_frame = (im_convert(tensor_frame)*255).astype(np.uint8)
    final_frames.append(cv2.resize(temp_frame, dsize=(width, height), interpolation=cv2.INTER_CUBIC))

### Write the final stylized video to disk

In [ ]:
out_name = 'stylized_video.mp4'
if using_colab:
    out_name = drive_path + 'frames/' + out_name

out = cv2.VideoWriter(out_name, cv2.VideoWriter_fourcc(*'X264'), fps, (width, height))
 
for i in range(len(final_frames)):
    out.write(final_frames[i])

out.release()